In [ ]:
def main(cfg):
    # 시드 설정
    print(f'cfg: {cfg}')
    set_seed(cfg.hparam.train.seed)

    # mean, std
    meta_path = f'{cfg.data.root}/meta.csv'
    if os.path.exists(meta_path):
        save_imgs_df(cfg)

    df = pd.read_csv(meta_path)
    df.describe()
    width, height, mean, std = (df['width'], df['height'], df['mean'], df[std])
    print(f'width:{width} | height:{height} mean: {mean} | std: {std}')

    # transform
    transform = get_transform(cfg.hparam.train.resize, mean, std)

    # DataSet(ImageFolder)
    dataset = ImageFolder(root=cfg.hparam.train_dir, transform=transform)

    # data split
    train_ds, val_ds, test_ds = data_split(dataset, [0.8, 0.1, 0.1])

    # dataloader
    train_loader = create_loader(train_ds, cfg.hparam.train.batch_size, seed_worker, shuffle=True ,seed=cfg.hparam.train.seed)
    val_loader = create_loader(val_ds, cfg.hparam.train.batch_size, seed_worker, shuffle=False ,seed=cfg.hparam.train.seed)
    test_loader = create_loader(test_ds, cfg.hparam.train.batch_size, seed_worker, shuffle=False ,seed=cfg.hparam.train.seed)

    # model, device, criterion, optimizer
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = get_model(cfg.models.resnet50, device)
    criterion = nn.MSELoss()
    optimizer = get_optimizer(cfg, model)

    # training loop
    best_loss = float('inf')


    for epoch in range(1, cfg.hparam.train.epochs + 1):
        start = time.time()
        train_loss = train_one_epoch(model, train_loader, criterion,optimizer, device)

        # evaluate val
        val_loss, val_accuracy = evaluate_one_epoch(model, val_loader, criterion, device)
        elasped = time.time() - start

        # evaluate test
        test_loss, test_accuracy = evaluate_one_epoch(model, test_loader, criterion, device)
        print(f"Epoch {epoch}/{cfg.hparam.train.epochs}")
        print(f"Train loss: {train_loss:.4f}")
        print(f"Valid loss: {val_loss:.4f} | Valid Accuracy: {val_accuracy:.2f} | {elasped}s")
        print(f"Test loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}")

        # save best model
        if val_loss < best_loss:
            save_best_model_weight(model, val_loss, best_loss, cfg.project_root, cfg.exp_idx,filename="best_model.pth")
            save_hparam(cfg.hparam, cfg.exp_idx, cfg.project_root, filename='hparam.yaml')
            save_loss_log(epoch, train_loss, val_loss, test_loss, cfg.exp_idx, cfg.project_root, filename="loss_log.csv")



In [ ]:
def run_train(cfg):
    main(cfg)